In [164]:
import numpy as np
import os,sys
import yaml
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import cv2
import matplotlib.animation as animation

In [2]:
def wavefront_initialize(pixelsize_h=1e-6,pixelsize_v=1e-6,npixels_h=2048,npixels_v=2048,amplitude_value=0.0):
    #
    #create array at object (aperture) plane
    #
    amplitude = np.zeros((npixels_h,npixels_v))  # amplitude map
    amplitude += amplitude_value
    p_i_h = np.arange(npixels_h) * pixelsize_h
    p_x = (p_i_h - 0.5 * (p_i_h[-1] - p_i_h[0]) )
    p_i_v = np.arange(npixels_v) * pixelsize_v
    p_y = (p_i_v - 0.5 * (p_i_v[-1] - p_i_v[0]) )
    return p_x,p_y,amplitude

In [3]:
def wavefront_aperture(p_x,p_y,amplitude,diameter_h=40e-6,diameter_v=40e-6,diameter_r=40e-6,type=0):
    # aperture_type: 0=circular, 1=Square
    p_xx = p_x[:, np.newaxis]
    p_yy = p_y[np.newaxis, :]

    filter = np.zeros_like(amplitude)
    if type == 0:  # Circular aperture
        radius = (diameter_r/2)
        print("radius=%f um"%(1e6*radius))
        filter_illuminated_indices = np.where(p_xx**2 + p_yy**2 < radius**2)
        if filter_illuminated_indices[0].size ==0:
            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")
        else:
            filter[filter_illuminated_indices] = 1.0
    elif type == 1:  # square
        radius_h = (diameter_h/2)
        radius_v = (diameter_v/2)
        print("radius_h=%f um,radius_v=%f um"%(1e6*radius_h,1e6*radius_v))
        filter_illuminated_indices = np.where( (np.abs(p_xx) < radius_h) & (np.abs(p_yy) < radius_v))
        if filter_illuminated_indices[0].size ==0:
            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")
        else:
            filter[filter_illuminated_indices] = 1.0
    else:
        raise ValueError("Aperture type (shape) not valid")

    return p_x,p_y,amplitude*filter

In [4]:
with open('configure.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
#print(conf_para)
alpha_x = conf_para['Lens']['alpha_x']
alpha_y = conf_para['Lens']['alpha_y']
focus_x = conf_para['Lens']['focus_x']
focus_y = conf_para['Lens']['focus_y']

det_dist = conf_para['Exp_geom']['det_dist']
defocus = conf_para['Exp_geom']['defocus']
ap_x = conf_para['Lens']['ap_x']
ap_y = conf_para['Lens']['ap_y']

ss_size = conf_para['Detector']['ss_size']
fs_size = conf_para['Detector']['fs_size']

pixelsize_x = conf_para['Detector']['pixelsize_x']
pixelsize_y = conf_para['Detector']['pixelsize_y']
wl = conf_para['Source']['wl']
k = 2*np.pi/wl

In [142]:
p_x,p_y,pupil_input = wavefront_initialize(pixelsize_h=5e-7,pixelsize_v=5e-7,npixels_h=1024,npixels_v=1024,amplitude_value=1.0)
p_x,p_y,pupil_lens = wavefront_aperture(p_x,p_y,pupil_input,diameter_h=40e-6,diameter_v=40e-6,type=1)

radius_h=20.000000 um,radius_v=20.000000 um


In [143]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
p3 = axes[0].imshow(np.abs(pupil_lens),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude_focus', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(pupil_lens)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase_focus', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [189]:
pupil_focus = np.fft.fftshift(np.fft.fft2(pupil_lens))
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
p3 = axes[0].imshow(np.abs(pupil_focus),extent = [x_min,x_max,y_min,y_max],norm=LogNorm())
axes[0].set_title('Amplitude_focus', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(pupil_focus)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase_focus', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('focus point.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [194]:
f = focus_x
phi = alpha_x*1e9*(pxy[1]/f)**3+alpha_y*1e9*(pxy[0]/f)**3+np.pi/wl/f*(pxy[1]**2+pxy[0]**2)
W = np.pi/wl/f*(pxy[1]**2+pxy[0]**2)+phi
pupil_lens_abe = pupil_lens*np.exp(1.j*(phi))
pupil_focus = np.fft.fftshift(np.fft.fft2(pupil_lens_abe))
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
p3 = axes[0].imshow(np.abs(pupil_lens_abe),extent = [x_min,x_max,y_min,y_max],norm=LogNorm())
axes[0].set_title('Amplitude_focus', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(pupil_lens_abe)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase_focus', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('focus point.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [202]:
dis = defocus*5000
f = focus_x
pxy = np.array(np.meshgrid(p_y,p_x))
phi = alpha_x*1e9*(pxy[1]/f)**3+alpha_y*1e9*(pxy[0]/f)**3+np.pi/wl/f*(pxy[1]**2+pxy[0]**2)
W = np.pi/wl/f*(pxy[1]**2+pxy[0]**2)
pupil = pupil_lens*np.exp(1.j*(W))
F_pupil_def = np.fft.fft2(pupil)


In [203]:
%matplotlib widget
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
plt.imshow(np.abs(F_pupil_def)**2,extent = [x_min,x_max,y_min,y_max],norm=LogNorm())
plt.colorbar()
plt.savefig('defocus point.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …